In [41]:
import pandas as pd
import os
import glob

all_matches = glob.glob(os.path.join('data/matches_upload/', "*.csv")) 
# In one line concatenate and drop duplicates of all matches files
match_uploads = pd.concat(pd.read_csv(f) for f in all_matches).drop_duplicates()
match_uploads.to_csv('data/matches.csv')

all_teams = glob.glob(os.path.join('data/team_upload/', "*.csv"))
team_uploads = pd.DataFrame()
for files in all_teams:
    data = pd.read_csv(files) 
    # add upload date from filename by splitting around parenthesis
    data['upload_date'] = files.split('(', 1)[1].split(')', 1)[0]
    team_uploads = pd.concat([team_uploads, data])  # Concatenate the DataFrames 
team_uploads.to_csv('data/teams.csv')